In [1]:
from clickhouse_driver import Client
import pandas as pd
import numpy as np

from datetime import datetime as dt
import datetime
from dateutil.relativedelta import relativedelta

from scipy.stats import entropy, skew as skewness, kurtosis

from tqdm import tqdm_notebook
import tqdm

import os

In [2]:
window = 10

p = 'BTCUSDT' 

n_days = 21

start_date = datetime.date(2023, 4, 3)

start_dates = [start_date + relativedelta(days = i) for i in range(n_days + 1)]
start_dates = [i.strftime('%Y_%m_%d') for i in start_dates]

## LAST TRADES

In [54]:
def generate_timestamps(df, time_col, window):
    
    start, end = df[time_col].min(), df[time_col].max()

    delta = relativedelta(end, start)
    delta = (
             delta.days * 24 * 60 * 60 + 
             delta.hours * 60 * 60 + 
             delta.minutes * 60 + 
             delta.seconds
            ) 

    ndelta = int(delta // window)

    timestamps = [start + relativedelta(seconds = i * window) for i in range(ndelta + 1)]
    timestamps = [datetime.datetime(i.year, i.month, i.day, i.hour, i.minute, i.second) for i in timestamps]
    
    return timestamps

def resample(df, time_col, timestamps):
    
    timestamps = pd.DataFrame({time_col: timestamps})
    
    df = df.merge(timestamps, how = 'outer').sort_values(by = time_col, ignore_index = True)
    
    prices = ['last_price', 'wavg_price']
    df[prices] = df[prices].fillna(method = 'ffill')
    
    df = df.fillna(0)
    
    return df

In [ ]:
for s in tqdm.notebook.tqdm(start_dates):
    print(s)
    try:
        last_trade = pd.read_csv(f'LAST_TRADES_{p}_{s}.csv')
        
        last_trade.sort_values(by = 'event_time', inplace = True)

        last_trade['event_time_r'] = last_trade['event_time'].apply(lambda x: dt(x.year, x.month, x.day, x.hour, x.minute,
                                                                         x.second // window * window))

        last_trade['return'] = last_trade['price'].pct_change() * 100
        last_trade['return_sq'] = last_trade['return'] ** 2

        last_trade['return_pos'] = last_trade['return'].apply(lambda x: x > 0).astype(int)
        last_trade['return_neg'] = last_trade['return'].apply(lambda x: x < 0).astype(int)
        
        last_trade['return_abs'] = last_trade['return'].apply(abs)

        wavg_price = last_trade.groupby('event_time_r').apply(lambda x: sum(x['price'] * x['quantity']) / 
                                                          sum(x['quantity'])).reset_index(name = 'wavg_price')

        last_price = last_trade.groupby('event_time_r')['price'].last().reset_index(name = 'last_price')

        last_volume = last_trade.groupby('event_time_r')['quantity'].last().reset_index(name = 'last_volume')
        trade_volume = last_trade.groupby('event_time_r')['quantity'].sum().reset_index(name = 'trade_volume')

        n_pos_returns = last_trade.groupby('event_time_r')['return_pos'].sum().reset_index(name = 'n_pos_returns')
        n_neg_returns = last_trade.groupby('event_time_r')['return_neg'].sum().reset_index(name = 'n_neg_returns')
        n_uniq_trades = last_trade.groupby('event_time_r')['tradeid'].nunique().reset_index(name = 'n_trades')

        rv = (last_trade.groupby('event_time_r')['return_sq'].sum() ** 0.5).reset_index(name = 'rv')
    
        skew = last_trade.groupby('event_time_r')['return'].apply(skewness).reset_index(name = 'return_skew')
        kurt = last_trade.groupby('event_time_r')['return'].apply(kurtosis).reset_index(name = 'return_kurt')

        df = (wavg_price.merge(last_price).merge(last_volume).merge(trade_volume).merge(n_pos_returns).merge(n_neg_returns).
              merge(n_uniq_trades).merge(rv).merge(skew).merge(kurt))

        df['pos_prevail'] = np.where(df['n_pos_returns'] > df['n_neg_returns'], 1, 0)

        timestamps = generate_timestamps(df, 'event_time_r', window)

        df_rs = resample(df, 'event_time_r', timestamps)

        df_rs['return_wavg'] = df_rs['wavg_price'].pct_change() * 100
        df_rs['return_last'] = df_rs['last_price'].pct_change() * 100

        df_rs['return_wavg_pos'] = np.where(df_rs['return_wavg'] > 0, 1, 0)
        df_rs['return_wavg_neg'] = np.where(df_rs['return_wavg'] < 0, 1, 0)

        df_rs['last_volume_delta'] = df_rs['last_volume'].diff()
        df_rs['trade_volume_delta'] = df_rs['trade_volume'].diff()

        df_rs['n_trades_delta'] = df_rs['n_trades'].diff()

        for i in df_rs.columns[5:]:
            for j in range(1, 4):
                df_rs[i + f'_l{j}'] = df_rs[i].shift(j)

        df_rs = df_rs.dropna()

        df_rs.drop(columns = ['wavg_price', 'last_price', 'last_volume', 'trade_volume', 'n_pos_returns', 'n_trades',
                              'n_neg_returns', 'return_skew', 'return_kurt', 'pos_prevail', 'n_trades_delta', 
                              'return_last', 'return_wavg', 'last_volume_delta',
                              'trade_volume_delta', 'return_wavg_pos', 'return_wavg_neg'], inplace = True)

        for_naming = df_rs['event_time_r'].min().date().strftime('%Y_%m_%d')

        df_rs.to_csv(f'LAST_TRADE_{p}_{for_naming}_{window}.csv', index = None)
    
    except Exception:
        print('error')